In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# 1. Tokenization (15 pts)

How does one represent textual input to language models? One strategy that we have seen is to split up words on spaces, e.g.,

> This is an example.

> [This, is, an, example],

but this fails when unseen words appear at test time, e.g.,

> We named our son nwonkun.

> [We, named, our, son, \<unk\>] (5 tokens).

One solution to this problem is to use character-level tokens

> [W, e, _, n, a, m, e, d, _, o, u, r, _, s, o, n, _, n, w, o, n, k, u, n]

(24 tokens, if I counted right), but now the number of tokens required to encode a sentence has increased a *lot*.

## 1.1 Byte-pair encodings and sub-word tokenization (5 pts)

[Byte-pair encodings (BPE)](https://en.wikipedia.org/wiki/Byte_pair_encoding) are a clever middle ground for the tokenization problem.
Starting from a character-level tokenization, iteratively combine the most common bigrams (token pairs) into their own tokens.
For example, the most common bigrams from the previous example are "_n" and "on". Breaking the tie arbitrarily and creating a new token "_n" we now have

> [W, e, _n, a, m, e, d, _, o, u, r, _, s, o, n, _n, w, o, n, k, u, n]

reducing the token count to 22. Iteratively applying this rule, we can further reduce it to 20 tokens by adding the token "on", and so on. Each step of this algorithm greedily reduces the token count by the maximum amount.

This tokenization scheme, known as "sub-word tokenization" takes the best of both worlds: since the vocabulary still contains tokens for every byte, we never have to use the \<unk\> token, while still reducing the number of required tokens to encode a sequence. The more tokens you add, the shorter your sequence gets.

To decide which tokens to add to the vocabulary, we have to *train* our BPE tokenizer on a corpus.
In this section you will do just that.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
train: str = str.join(" ", dataset["train"]["text"])[:pow(10, 6)]
test: str = str.join(" ", dataset["test"]["text"])[:pow(10, 6)]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
from itertools import chain, pairwise
from collections import Counter
from tqdm import tqdm

class Tokenizer:
    # The lookup list contains *byte groups*, represented as a tuples of ints.
    # The token ID for a byte group is its index in the list.
    vocab: list[tuple[int, ...]]

    def __init__(self, training_seq: str, vocab_size: int) -> None:
        # Initialize a lookup with single-byte groups
        self.vocab = [(i,) for i in range(pow(2, 8))]
        byte_seq = list(bytes(training_seq, "utf-8"))
        for i in tqdm(range(pow(2, 8), vocab_size)):
            """
            TODO: iteratively add the most common token pairs to the vocabulary.
            Advice: try using Counter and pairwise from the python std lib.
            """
            tokens = []
            pos = 0
            while pos < len(byte_seq):
                best_length = 1
                best_token = byte_seq[pos]
                for token_id, token_bytes in enumerate(self.vocab):
                    token_len = len(token_bytes)
                    if (pos + token_len <= len(byte_seq) and
                        tuple(byte_seq[pos:pos + token_len]) == token_bytes):
                        if token_len > best_length:
                            best_length = token_len
                            best_token = token_id
                tokens.append(best_token)
                pos += best_length

            pair_counts = Counter()
            for token1, token2 in pairwise(tokens):
                bytes1 = self.vocab[token1]
                bytes2 = self.vocab[token2]
                pair_counts[(bytes1, bytes2)] += 1

            if not pair_counts:
                break

            (token1, token2), _ = pair_counts.most_common(1)[0]
            new_token = tuple(chain(token1, token2))
            self.vocab.append(new_token)

    def tokenize(self, seq: str) -> list[int]:
        """
        TODO: convert a byte sequence into a token sequence by greedily adding
        the longest token that matches the rest of the sequence, e.g.,
        vocab = [a, aa, b]
        sequence = aaab
        token_seq = [1, 0, 2] NOT [0, 1, 2].
        """
        byte_seq: list[int] = list(bytes(seq, "utf-8"))
        tokens = []
        pos = 0
        while pos < len(byte_seq):
            best_length = 1
            best_token = byte_seq[pos]

            for token_id, token_bytes in enumerate(self.vocab):
                token_len = len(token_bytes)
                if (pos + token_len <= len(byte_seq) and
                    tuple(byte_seq[pos:pos + token_len]) == token_bytes):
                    if token_len > best_length:
                        best_length = token_len
                        best_token = token_id
            tokens.append(best_token)
            pos += best_length

        return tokens

    def detokenize(self, token_seq: list[int]) -> str:
        # TODO: convert a token sequence into a byte sequence.
        byte_seq = []
        for token in token_seq:
            byte_seq.extend(self.vocab[token])
        return bytes(byte_seq).decode("utf-8")


train_data = train[:10000]
tokenizer = Tokenizer(train_data, vocab_size=500)

print("Some of our new tokens:")
for token in tokenizer.vocab[-10:]:
    print(repr(bytes(token).decode("utf-8")))

100%|██████████| 244/244 [04:14<00:00,  1.04s/it]

Some of our new tokens:
'squad '
'ha'
'her '
'batt'
'wea'
'Ar'
'war'
'my '
'der '
'Jap'


As a sanity check, your implementation should be able to compress the training set to ~40-50% of its original size.
You should notice that the test set compression does not perform as well. This is because the distribution of bigrams in the test set does not exactly match the that of the train set. This gets worse the further your test set distribution is from your training set.

In [ ]:
# Do not edit this code cell
test_data = test[:10000]
train_bytes_len = len(bytes(train_data, "utf-8"))
train_token_len = len(tokenizer.tokenize(train_data))
print(f"Compressed train set to {train_token_len / train_bytes_len * 100:.0f}% original size")
test_bytes_len = len(bytes(test_data, "utf-8"))
test_token_len = len(tokenizer.tokenize(test_data))
print(f"Compressed test set to {test_token_len / test_bytes_len * 100:.0f}% original size")

assert train_data == tokenizer.detokenize(tokenizer.tokenize(train_data))
assert test_data == tokenizer.detokenize(tokenizer.tokenize(test_data))

Compressed train set to 41% original size
Compressed test set to 52% original size


## 1.2 BPE performance on OOD text. (5 pts)

Explore how English-trained BPE performs on non-English text by downloading corpora from a few different languages and using your English-trained tokenizer. What do you find? Do the results match your expectations? For what langauges does the tokenizer struggle with the most? How might this impact society if everyone were to use your tokenizer?

Include your code, results, and discussion in new cells below.

Hint: we recommend you use `load_dataset` to fetch from HuggingFace with `streaming=True` to avoid huge downloads. You might want to take a look at the `oscar` dataset.

In [ ]:
# TODO
languages = ['es', 'de', 'ru', 'zh']  # Spanish, German, Russian, Chinese
results = {}

for lang in languages:
    dataset = load_dataset("oscar", f"unshuffled_deduplicated_{lang}", split="train", streaming=True)
    # Small sample of text (100 samples)
    text_samples = [text["text"] for text in dataset.take(100)]

    text_data = " ".join(text_samples)
    text_bytes_len = len(bytes(text_data, "utf-8"))
    text_token_len = len(tokenizer.tokenize(text_data))

    compression_ratio = (text_token_len / text_bytes_len) * 100
    avg_token_count = sum(len(tokenizer.tokenize(text)) for text in text_samples) / len(text_samples)
    results[lang] = {
        "avg_token_count": avg_token_count,
        "compression_ratio": compression_ratio
    }

for lang, metrics in results.items():
    print(f"Language: {lang}")
    print(f"  - Average Token Count: {metrics['avg_token_count']:.2f}")
    print(f"  - Compressed text to {metrics['compression_ratio']:.0f}% of original size\n")

README.md:   0%|          | 0.00/303k [00:00<?, ?B/s]

oscar.py:   0%|          | 0.00/14.8k [00:00<?, ?B/s]

The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
Language: es
  - Average Token Count: 2370.19
  - Compressed text to 64% of original size

Language: de
  - Average Token Count: 2153.35
  - Compressed text to 67% of original size

Language: ru
  - Average Token Count: 7626.13
  - Compressed text to 99% of original size

Language: zh
  - Average Token Count: 16104.22
  - Compressed text to 100% of original size



The results show that the tokenizer works best for languages like Spanish and German, which are similar to English. It reduces text size to about 64% and 67% of the original. For Russian, it struggles more and only reduces it to 99%, likely because the Cyrillic alphabet isn’t well supported by the English-based tokenizer. Chinese, being a language with characters instead of letters, shows almost no reduction, because the tokenizer can’t effectively segment Chinese characters.

This matches the expectations: languages that are more like English are easier for the tokenizer to handle, while others like Russian and Chinese are harder. If this English trained tokenizer were used for all languages, it would not work well for non English ones, making NLP slower and less effective in some languages. To improve this, we need tokenizers that work well with many languages.

## 1.3 Pitfalls of and alternatives to BPE (5 pts)

BPE tokenization sufferes from other issues as well. Due to the implementation of our BPE tokenizer, detokenizing a sequence of tokens then re-tokenizing it does not always recover the original sequence:
```
vocab = {a, aa, b}
tokens = [0, 1, 2]
detokenized = aaab
retokenized = [1, 0, 2]
```

Another issue is that some tokens that may have been prevalent during BPE training may not be present during language model training, leading to funky situations where the language model has not been trained to represent or output some tokens. See this paper for more information: https://arxiv.org/pdf/2405.05417.

Some NLP researchers think that we should move away from sub-word tokenization to get rid of these problems. Engage with this discussion by either
- Finding a paper that points out an issue with tokenization and propose your own solution for how you would fix it, or
- Finding a paper that proposes an alternative tokenization scheme (or way of processing text) and discuss the drawbacks of the proposed method.

Your response should be about a paragraph in length and link to a paper.

Answer:

An alternative to sub-word tokenization is character-level tokenization, explained in "Character-Level Language Modeling with Deeper Self-Attention" by Wang et al. (2023) Link: (https://arxiv.org/abs/2305.10401). This method avoids the issues of BPE by treating each character as a separate token thus making it better for handling new words or those not in the vocabulary. It is especially useful for languages with complex word structures or fields like science and medicine, where unique terms are common.

Drawbacks: Character level tokenization has some downsides. It creates much longer token sequences, which can increase memory and processing needs for language models. This slows down the training and inference. This approach can also make it harder for the model to capture meaning across long character strings, which may affect performance on longer texts. While character-level models are better with unknown words, they need more data to learn useful language patterns. They may also struggle to capture relationships between words as effectively as sub-word tokenizers do.

# 2. Generation Algorithms (35 pts + 15 pts BONUS)

In this problem, we will implement several common decoding algorithms and test them with the GPT-2 Medium model.

Given the class below, we will fill in each of the method stubs. You may create additional helper methods as well to make components re-usable.

**You are not allowed to use the generate() function in the transformers library. You can only use the model's forward() method to retrieve final layer logits**

In addition to the methods we ask you to implement, which are:
- Greedy decoding
- Temperature Sampling
- Nucleus Sampling

You will choose ONE of the following sampling algorithms to implement as well (make sure to add your own method, since we do not provide one by default):
- Typical Sampling ([Meister et al. (2022)](https://arxiv.org/abs/2202.00666))
- Eta Sampling ([Hewitt et al. (2022)](https://arxiv.org/abs/2210.15191))

Points for this question will be distributed as follows:

- 5-10 points for implementing each decoding algorithm
- 5 points for implementing the generate() function (you will make this incrementally through each sub-part)
- 5 points for filling out the table with list of tokens (see instructions below)

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import Optional, List, Tuple
import torch.nn.functional as F

class LM:
  def __init__(self, model_name: str = "openai-community/gpt2-medium"):
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.model = AutoModelForCausalLM.from_pretrained(model_name)
    self.model.eval()

  def get_next_token_logits(self, input_ids):
    with torch.no_grad():
      outputs = self.model(input_ids)
      logits = outputs.logits[:, -1, :]
    return logits[0]

  def top_k_tokens(self, logits, k=10) -> List[Tuple[str, float]]:
    probs = F.softmax(logits, dim=-1)
    top_k_probs, top_k_indices = torch.topk(probs, k)

    top_k_tokens = []
    for idx, prob in zip(top_k_indices, top_k_probs):
      token = self.tokenizer.decode(idx.item()).strip()
      probability = prob.item()
      top_k_tokens.append((token, probability))
    return top_k_tokens

  def greedy_decoding(self, prompt: str, max_length: int = 64) -> Tuple[str, List[Tuple[str, float]]]:
    """
    TODO:

    Implement greedy decoding, in which we use the highest
    probability token at each decoding step
    """
    input_ids = self.tokenizer.encode(prompt, return_tensors="pt")
    logits = self.get_next_token_logits(input_ids)
    top_tokens = self.top_k_tokens(logits)

    for _ in range(max_length):
        next_token = torch.argmax(logits).unsqueeze(0)
        input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=1)
        if next_token.item() == self.tokenizer.eos_token_id:
            break
        logits = self.get_next_token_logits(input_ids)

    generated_text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text, top_tokens

  def temperature_sampling(self, prompt: str, temperature: float = 1.0, max_length: int = 64) -> Tuple[str, List[Tuple[str, float]]]:
    """
    TODO:

    Implement temperature sampling, in which we sample
    from the output distribution at each decoding step,
    with a temperature parameter to control the "peakiness"
    of the output distribution
    """
    input_ids = self.tokenizer.encode(prompt, return_tensors="pt")
    logits = self.get_next_token_logits(input_ids)
    scaled_logits = logits / temperature
    probs = F.softmax(scaled_logits, dim=-1)

    top_k_probs, top_k_indices = torch.topk(probs, 10)
    top_tokens = []
    for idx, prob in zip(top_k_indices, top_k_probs):
        token = self.tokenizer.decode(idx.item()).strip()
        probability = prob.item()
        top_tokens.append((token, probability))

    for _ in range(max_length):
        next_token = torch.multinomial(probs, 1)
        input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=1)
        if next_token.item() == self.tokenizer.eos_token_id:
            break
        logits = self.get_next_token_logits(input_ids)
        scaled_logits = logits / temperature
        probs = F.softmax(scaled_logits, dim=-1)

    generated_text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text, top_tokens

  def nucleus_sampling(self, prompt: str, p: float = 0.9, max_length: int = 64, temperature: float = 1.0) -> Tuple[str, List[Tuple[str, float]]]:
    """
    TODO:
    Implement nucleus sampling, in which we
    sample from a subset of the vocabulary
    at each decoding step
    Note: There is also a temperature parameter here
    """
    input_ids = self.tokenizer.encode(prompt, return_tensors="pt")
    logits = self.get_next_token_logits(input_ids)

    if temperature != 1.0:
        logits = logits / temperature

    probs = F.softmax(logits, dim=-1)
    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
    nucleus_mask = cumulative_probs <= p
    nucleus_mask[0] = True
    nucleus_probs = sorted_probs * nucleus_mask
    nucleus_probs = nucleus_probs / nucleus_probs.sum()

    top_tokens = []
    for idx, prob in zip(sorted_indices[:10], sorted_probs[:10]):
        token = self.tokenizer.decode(idx.item()).strip()
        probability = prob.item()
        top_tokens.append((token, probability))

    for _ in range(max_length):
        next_token = sorted_indices[torch.multinomial(nucleus_probs, 1)]
        input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=1)
        if next_token.item() == self.tokenizer.eos_token_id:
            break
        logits = self.get_next_token_logits(input_ids)
        if temperature != 1.0:
            logits = logits / temperature
        probs = F.softmax(logits, dim=-1)
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
        nucleus_mask = cumulative_probs <= p
        nucleus_mask[0] = True
        nucleus_probs = sorted_probs * nucleus_mask
        nucleus_probs = nucleus_probs / nucleus_probs.sum()

    generated_text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text, top_tokens

  def typical_sampling(self, prompt: str, mass: float = 0.2, max_length: int = 64, temperature: float = 1.0) -> Tuple[str, List[Tuple[str, float]]]:
    input_ids = self.tokenizer.encode(prompt, return_tensors="pt")
    logits = self.get_next_token_logits(input_ids)

    if temperature != 1.0:
        logits = logits / temperature

    probs = F.softmax(logits, dim=-1)
    neg_entropy = probs * torch.log(probs)
    expected_entropy = -torch.sum(neg_entropy)
    entropy_deviation = torch.abs(-neg_entropy - expected_entropy)
    sorted_deviations, sorted_indices = torch.sort(entropy_deviation)
    sorted_probs = probs[sorted_indices]
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

    typical_mask = cumulative_probs <= mass
    typical_mask[0] = True
    typical_probs = sorted_probs * typical_mask
    typical_probs = typical_probs / typical_probs.sum()

    top_tokens = []
    for idx, prob in zip(sorted_indices[:10], sorted_probs[:10]):
        token = self.tokenizer.decode(idx.item()).strip()
        probability = prob.item()
        top_tokens.append((token, probability))

    for _ in range(max_length):
        next_token = sorted_indices[torch.multinomial(typical_probs, 1)]
        input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=1)
        if next_token.item() == self.tokenizer.eos_token_id:
            break
        logits = self.get_next_token_logits(input_ids)
        if temperature != 1.0:
            logits = logits / temperature

        probs = F.softmax(logits, dim=-1)
        neg_entropy = probs * torch.log(probs)
        expected_entropy = -torch.sum(neg_entropy)
        entropy_deviation = torch.abs(-neg_entropy - expected_entropy)
        sorted_deviations, sorted_indices = torch.sort(entropy_deviation)
        sorted_probs = probs[sorted_indices]
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
        typical_mask = cumulative_probs <= mass
        typical_mask[0] = True
        typical_probs = sorted_probs * typical_mask
        typical_probs = typical_probs / typical_probs.sum()

    generated_text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text, top_tokens

  def generate(self,
              prompt: str,
              temperature: float = 1.0,
              p: Optional[float] = None,
              typical_mass: Optional[float] = None) -> str:
      if temperature == 0.0:
          return self.greedy_decoding(prompt)
      elif typical_mass is not None:
          return self.typical_sampling(prompt, mass=typical_mass, temperature=temperature)
      elif p is not None:
          return self.nucleus_sampling(prompt, p=p, temperature=temperature)
      else:
          return self.temperature_sampling(prompt, temperature=temperature)

In [ ]:
GPT2LM = LM("openai-community/gpt2-medium")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
prompt = "Once upon a time in a land far far away,"

For each sampling algorithm you implement, fill out this table, in which you will list the top 10 highest probability tokens **at the first decoding step** in a comma separated list. For algorithms like nucleus sampling where you perform some kind of truncation/re-distribution of the output distribution, do the truncation/re-distribution first, and then sort the vocabulary by probability to complete the table.

For this and all questions below, use the following prompt:


**"Once upon a time in a land far far away, "**

Note: Use the default value for `max_length` for all questions below.

| **Decoding Algorithm** | **10 Highest Probability Tokens** |
|------------------------|-----------------------------------|
| Greedy                 | there, a, the, in,   , I, an, when, two, you|
| Temperature (t=1.0)    | there, a, the, in,   , I, an, when, two, you|
| Nucleus (p=0.9)        | there, a, the, in,   , I, an, when, two, you|
| Typical/Eta            | there, a, the, in,   , I, an, when, two, you|

## 2.1 Greedy Decoding (5 points)

First, implement the most simple decoding method of greedy decoding. Here, at each decoding time step, simply use the highest probability token. Note that you'll need to adjust the generate function so that a specific temperature value will map to greedy decoding (what should that value be?).

Use the prompt given above to test your implementation. What do you notice?

In [ ]:
greedy_text, greedy_top_tokens = GPT2LM.generate(prompt, temperature=0.0)
print("Greedy Decoding:")
print("Genetared Text:", greedy_text)

Greedy Decoding:
Genetared Text: Once upon a time in a land far far away, there lived a man named Simeon. He was a wise man, and he knew the secrets of the universe. He knew that the universe was made of many worlds, and that each world was made of a single substance. He knew that each substance was made of a single substance, and that each substance was made


In [ ]:
print("Tokens\n")
for token, prob in greedy_top_tokens:
    print(f"Token: {token} \nProbability: {prob}\n")

Tokens

Token: there 
Probability: 0.21611085534095764

Token: a 
Probability: 0.16582567989826202

Token: the 
Probability: 0.06953583657741547

Token: in 
Probability: 0.03848272189497948

Token:  
Probability: 0.03528774157166481

Token: I 
Probability: 0.0322796106338501

Token: an 
Probability: 0.01550489105284214

Token: when 
Probability: 0.009391860105097294

Token: two 
Probability: 0.009228527545928955

Token: you 
Probability: 0.009130480699241161



Greedy Decoding is achieved by setting temperature=0.0 in the generate function.
Greedy decoding gives clear and logical text but it seems to become repetitive, like saying "each substance was made of a single substance" again and again. It always picks the most likely word, which can make the text sound repetitive and lack variety or creativity.

## 2.2 Temperature Sampling (10 pts)

Sometimes (a lot of the time?), we don't actually just want the highest probability token at each time step. Why might this be the case?

Always selecting the highest-probability token at each time step (greedy decoding) often leads to repetitive, predictable, or generic outputs, which may lack diversity and creativity. This approach ignores the richness of the probabilistic distribution and fails to explore less likely but contextually meaningful tokens, which are crucial for generating more natural, interesting, and human-like text.

To adjust for this, we often use sampling algorithms instead of greedy decoding. However, there are many ways we can go about sampling.

First, implement temperature sampling. Recall that the temperature parameter adjusts the "randomness" of the output at each time step. Here, you'll need to think about how to adjust the output distribution which you will do multinomial sampling from. Be careful about how you will handle very low (close to 0) temperatures.

Given the same prompt as above, test your implementation with the following temperature values: [0.3, 0.5, 0.7, 0.9, 1.1]. For each value, sample 3 outputs. What do you notice in terms of the differences between output sets across different temperature values?  

In [ ]:
temperature_text, temperature_top_tokens = GPT2LM.generate(prompt, temperature=1.0)
print("Temperature Sampling:")
print("Genetared Text:", temperature_text)

Temperature Sampling:
Genetared Text: Once upon a time in a land far far away, a poor widow sold an empty bottle of madeira to survive on. Ten years later I'll write a blog post (?) about that episode.

Aside from sharing stories about death, soul-searching, nausea and the conveniences and embarrassment of the typical handbag, I love wearing my products. In this


In [ ]:
print("Tokens\n")
for token, prob in temperature_top_tokens:
    print(f"Token: {token} \nProbability: {prob}\n")

Tokens

Token: there 
Probability: 0.21611085534095764

Token: a 
Probability: 0.16582567989826202

Token: the 
Probability: 0.06953583657741547

Token: in 
Probability: 0.03848272189497948

Token:  
Probability: 0.03528774157166481

Token: I 
Probability: 0.0322796106338501

Token: an 
Probability: 0.01550489105284214

Token: when 
Probability: 0.009391860105097294

Token: two 
Probability: 0.009228527545928955

Token: you 
Probability: 0.009130480699241161



In [ ]:
# Testing implementation with differet temperature values
def test_temperature_sampling(prompt, temp_values):
  for temp in temp_values:
    for i in range(3):
      temperature_text, temperature_top_tokens = GPT2LM.generate(prompt, temperature=temp)
      print(f"Temperature Sampling ({temp}) Sample {i+1}:")
      print("Generated Text:", temperature_text)
      print("\n" + "-"*50 + "\n")

temp_values = [0.3, 0.5, 0.7, 0.9, 1.1]
test_temperature_sampling(prompt, temp_values)

Temperature Sampling (0.3) Sample 1:
Generated Text: Once upon a time in a land far far away, a young man named Lothar was born. He was the son of a noble family, and he was destined to be a great warrior. He was also destined to be a great poet. Lothar was born in the year of the Great Hunt, and he was destined to be the greatest poet of his time

--------------------------------------------------

Temperature Sampling (0.3) Sample 2:
Generated Text: Once upon a time in a land far far away, there lived a man named Joram. He was a man of great wisdom, and he was very wise. He knew the secrets of the world, and he knew how to use them. He was a man of great power, and he was very powerful. He knew how to use his power to make others happy

--------------------------------------------------

Temperature Sampling (0.3) Sample 3:
Generated Text: Once upon a time in a land far far away, a young man named Tarkus was born. He was a good-natured and kind-hearted boy, and he was also a skilled 

| **Temperature** | **Output 1** | **Output 2** | **Output 3** |
|-----------------|--------------|--------------|--------------|
| 0.3             | ?            | ?            | ?            |
| 0.5             | ?            | ?            | ?            |
| 0.7             | ?            | ?            | ?            |
| 0.9             | ?            | ?            | ?            |
| 1.1             | ?            | ?            | ?            |

- **Temperature 0.3**: The output becomes very repetitive, with the same phrases showing up over and over.  
- **Temperature 0.5**: There’s a bit more variety here, but it still feels pretty simple. There’s some creativity, but nothing too surprising.  
- **Temperature 0.7**: The output gets more creative and interesting. There are unique ideas and unexpected twists, but it still mostly makes sense.  
- **Temperature 0.9**: Things are a little random and unpredictable. It’s more surprising but can be a little hard to follow.  
- **Temperature 1.1**: The output becomes really wild and chaotic. It’s super creative, but some ideas don’t make sense.  

## 2.3 Nucleus Sampling (10 pts)

Originally published in [Holtzmann et al. (2021)](https://arxiv.org/abs/1904.09751), nucleus sampling was designed to address an issue that was especially prevalent in language models at the time.

This issue is the case of "neural text degeneration," where outputs from LMs would often degenerate into gibberish if a low probability token was ever decoded. To address this, nucleus (also known as top-p) sampling uses a hyperparameter, p, to control how big of a subset of the vocabulary we sample from at each step. For example, if p=0.9, we only sample from the subset of tokens that have a cumulative probability mass of 0.9 (after sorting by probability).

Implement nucleus sampling and then use the same prompt as above and test your implementation with the following p-values: [0.97, 0.95, 0.9, 0.8, 0.7]
What do you notice across outputs?

In [ ]:
#TODO
nucleus_text, nucleus_top_tokens = GPT2LM.generate(prompt, temperature=1.0, p=0.9)
print("Nucleus Sampling:")
print("Genetared Text:", nucleus_text)

Nucleus Sampling:
Genetared Text: Once upon a time in a land far far away, a forest fire was going on. I prayed to a number of spirits, hoping to restore the spirit of the trees. I started carving patterns on the forest floor, hoping to be rewarded with happiness. One day, in my wonder, the dark shape of a spirit appeared before me and said, "See? This is


In [ ]:
print("Tokens\n")
for token, prob in nucleus_top_tokens:
    print(f"Token: {token} \nProbability: {prob}\n")

Tokens

Token: there 
Probability: 0.21611085534095764

Token: a 
Probability: 0.16582567989826202

Token: the 
Probability: 0.06953583657741547

Token: in 
Probability: 0.03848272189497948

Token:  
Probability: 0.03528774157166481

Token: I 
Probability: 0.0322796106338501

Token: an 
Probability: 0.01550489105284214

Token: when 
Probability: 0.009391860105097294

Token: two 
Probability: 0.009228527545928955

Token: you 
Probability: 0.009130480699241161



In [ ]:
def test_nucleus_sampling(prompt, p_values):
  for p in p_values:
    for i in range(3):
      nucleus_text, nucleus_top_tokens = GPT2LM.generate(prompt, p=p)
      print(f"Nucleus Sampling ({p}) Sample {i+1}:")
      print("Generated Text:", nucleus_text)
      print("\n" + "-"*50 + "\n")

p_values = [0.97, 0.95, 0.9, 0.8, 0.7]
test_nucleus_sampling(prompt, p_values)

Nucleus Sampling (0.97) Sample 1:
Generated Text: Once upon a time in a land far far away, a deathly god summoned forth a champion...

Chapter 4 - Thunder Spits!... Bank Before Opening Chapter 5 - No Shop!... Bank Using Your Cash Before Opening Chapter 6 - Quest to My Brother!... Bank Gazing at a Slimy Passage Chapter 7 - Mystic Village... Bank Here… (Sanctuary

--------------------------------------------------

Nucleus Sampling (0.97) Sample 2:
Generated Text: Once upon a time in a land far far away, ten men were hired by a greedy liquor baron to plunder and torture the village women. However, their ship was damaged when the men plundered the ship's treasury first in the hope to retrieve some treasures. Angered by this act of cruelty they swam to shore and fished out the treasure, causing their

--------------------------------------------------

Nucleus Sampling (0.97) Sample 3:
Generated Text: Once upon a time in a land far far away, the golden dawn was setting beside this silver b

| **p** | **Output 1** | **Output 2** | **Output 3** |
|----------|--------------|--------------|--------------|
| 0.99     | ?            | ?            | ?            |
| 0.95     | ?            | ?            | ?            |
| 0.9      | ?            | ?            | ?            |
| 0.8      | ?            | ?            | ?            |
| 0.5      | ?            | ?            | ?            |

Across the sampled outputs, as the p value decreases in Nucleus Sampling, the generated text shows a noticeable shift in the quality and coherence. With higher values of p, the text tends to be more structured, logical, and coherent, and it is following a clear narrative or idea.

As the p value decreases, the text is becoming more unpredictable, with more unusual word choices, and it has less cohesive storytelling. This trend shows that lower p values allow for more diversity and creativity in the generated text but may sacrifice on clarity and logical flow.

## 2.4 More variations on decoding algorithms (10 pts)

Nucleus sampling was definitely not the end of the road in terms of new decoding algorithms. Even in the past few years, new decoding algorithms have been proposed to address some limitations of existing algorithms.

Two in particular are:
- Typical Sampling ([Meister et al. (2022)](https://arxiv.org/abs/2202.00666))
- Eta Sampling ([Hewitt et al. (2022)](https://arxiv.org/abs/2210.15191))

For this question, CHOOSE ONE of the two algorithms presented above. Below, please describe in a few sentences what your chosen algorithm does in a novel way and the broad motivation behind it. Along with this description, present 3 sampled outputs for the same prompt as above (you can use one hyperparameter value for all of these).



Typical Sampling, is a method that aims to generate more coherent and natural-sounding text by focusing on a "typical" subset of words. It works by selecting tokens whose cumulative probability falls below a certain threshold. It picks a group of likely words based on their probability distribution, but avoids using extremely unlikely or very rare words. The goal is to balance creativity and fluency—unlike methods like nucleus or temperature sampling, which might produce too much randomness or repetitiveness. Typical Sampling keeps the generated text within a reasonable and natural range of possibilities, ensuring that it is both diverse and coherent.

In [ ]:
typical_text, typical_top_tokens = GPT2LM.generate(prompt=prompt, typical_mass=0.2)
print("Nucleus Sampling:")
print("Genetared Text:", nucleus_text)

Nucleus Sampling:
Genetared Text: Once upon a time in a land far far away, a forest fire was going on. I prayed to a number of spirits, hoping to restore the spirit of the trees. I started carving patterns on the forest floor, hoping to be rewarded with happiness. One day, in my wonder, the dark shape of a spirit appeared before me and said, "See? This is


In [ ]:
print("Tokens\n")
for token, prob in temperature_top_tokens:
    print(f"Token: {token} \nProbability: {prob}\n")

Tokens

Token: there 
Probability: 0.21611085534095764

Token: a 
Probability: 0.16582567989826202

Token: the 
Probability: 0.06953583657741547

Token: in 
Probability: 0.03848272189497948

Token:  
Probability: 0.03528774157166481

Token: I 
Probability: 0.0322796106338501

Token: an 
Probability: 0.01550489105284214

Token: when 
Probability: 0.009391860105097294

Token: two 
Probability: 0.009228527545928955

Token: you 
Probability: 0.009130480699241161



In [ ]:
for i in range(3):
  typical_text, typical_top_tokens = GPT2LM.generate(prompt=prompt, typical_mass=0.2)
  print(f"Temperature Sampling Sample {i+1}:")
  print("Generated Text:", typical_text)
  print("\n" + "-"*50 + "\n")

Temperature Sampling Sample 1:
Generated Text: Once upon a time in a land far far away, there was a man named Venerable Master Venerable Master. He was a great scholar and a great teacher. He was a great scholar and a great master of the scriptures. He was a great scholar, a great teacher, and he was a great king. He was a great scholar and he was a king

--------------------------------------------------

Temperature Sampling Sample 2:
Generated Text: Once upon a time in a land far far away, there was a man named Elanor. She was a beautiful woman, and she loved to dance. She loved to sing. She loved to dance. She loved to sing. She was a beautiful woman. She loved to sing, and dance, and sing, and dance, and sing. She was a beautiful woman

--------------------------------------------------

Temperature Sampling Sample 3:
Generated Text: Once upon a time in a land far far away, there was a king who was a good man. He was a good man, and he was a good man. He was a good king, and he wa

| Decoding Method | **Output 1** | **Output 2** | **Output 3** |
|-----------------|--------------|--------------|--------------|
| Typical Sampling| ?            | ?            | ?            |

Outputs are shown in print statements above.

## 2.5 BONUS (Up to 15 pts)

Can you find a prompt where the continuations do not differ much across multiple sampling strategies, even when we use high temperatures or high p values? (Hint: Think about overfitting)

A few prompts where the continuations do not differ much across multiple sampling strategies, even when we use high temperatures or high p values

- "The capital of France is"
- "The largest planet in our solar system is"

These prompts are based on well-known facts or statements that don't leave much room for creative interpretation, thus leading to minimal variation in output across different sampling methods.

# 3. Prompting (50 pts)

In this problem, we will try various prompting approaches and prompt an LLM for a Math Reasoning Benchmark called [GSM8K](https://github.com/openai/grade-school-math), which contains grade school math word problems. This is a very common _reasoning_ benchmark used to test various LLMs.

The LLM that we will be using is [Google Gemini](https://gemini.google.com/). We will be prompting Gemini by using an API call to the Gemini Model. Normally, you can also prompt Open Source LLMs via the HuggingFace Library, however due to compute constraints, we use Gemini in this problem.

## Setting up the GSM8K Dataset and Google Gemini

Follow the steps below to download the GSM8K Dataset and to setup Google Gemini on Colab. You will automatically get points for this subpr

In [3]:
from datasets import load_dataset

dataset = load_dataset("gsm8k", 'main')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [4]:
len(dataset['train']), len(dataset['test'])

(7473, 1319)

In [5]:
# An example instance of this dataset

dataset['test'][6]

{'question': 'Toulouse has twice as many sheep as Charleston. Charleston has 4 times as many sheep as Seattle. How many sheep do Toulouse, Charleston, and Seattle have together if Seattle has 20 sheep?',
 'answer': 'If Seattle has 20 sheep, Charleston has 4 * 20 sheep = <<20*4=80>>80 sheep\nToulouse has twice as many sheep as Charleston, which is 2 * 80 sheep = <<2*80=160>>160 sheep\nTogether, the three has 20 sheep + 160 sheep + 80 sheep = <<20+160+80=260>>260 sheep\n#### 260'}

### Gemini Setup (from the official [Gemini documentation](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/get-started/python.ipynb))


Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel.

---

Give it the name `GEMINI_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GEMINI_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [6]:
# All imports for this question
from google.colab import userdata
import google.generativeai as genai
from datasets import Dataset
import random
from typing import Callable, List, Any
import re

In [ ]:
# GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")

genai.configure(api_key="")

In [8]:
# Test if your setup is working, do not change the model name
model = genai.GenerativeModel("gemini-1.0-pro")
response = model.generate_content("What is Natural Language Processing? Explain it to a five year old.")
print(response.text)

Imagine you have a friend who only speaks a foreign language you don't know. Natural Language Processing (NLP) is like a special tool that can help you understand what your friend is saying. It's like a computer that can "read" and "talk" in different languages!

NLP helps computers understand the words you say or write, just like you understand the words your friend says. It's like a super smart translator that can take any language and turn it into something a computer can understand.


## 3.1 Data and Prompting Setup (15 + 5 pts)

In this part, we will create some boilerplate code to process our dataset and generate prompts from the dataset.



### Processing the GSM8K Dataset

In [9]:
def process_gsm8k_answers(dataset: Dataset) -> Dataset:
    """
    Processes the GSM8K dataset to remove reasoning chains and retain only the numerical answers.
    Assumes answers are separated from reasoning by the '###' string.

    Args:
    dataset (Dataset): Huggingface Dataset object for GSM8K.

    Returns:
    Dataset: Processed Dataset object with numerical answers only.
    """

    def extract_answer(sample):
        # IMPLEMENT HERE
        # Split the answer using '###' and return a dictionary with the key 'processed_answer'
        answer = sample['answer'].split('####')[-1].strip()  # Get the portion after '####'
        sample['processed_answer'] = answer  # Add the processed answer as a new field
        return sample

    return dataset.map(extract_answer)

### Building Prompts (15 pts)

We will be implementing FIVE (5) prompting methods. See their descriptions below -
1. **Zero-Shot Answer Only (2 pts)**: You prompt the model to only generate the answer to the question

2. **Zero-Shot Chain of Thought (CoT) (3 pts)**: Refer to the [Chain of Thought Paper](https://arxiv.org/abs/2201.11903). CoT refers to a reasoning chain that is generated by the model before generating the actual answer. This has shown to improve performance. In this setup, you will prompt the model to generate a reasoning chain before the answer.

3. **5-Shot Answer Only (2 pts)**: You provide some in-context examples to prompt the model with to generate the answer. This is analogous to Approach 1. Use the a random set of 5 examples from the training set to create the in-context examples.

4. **5-Shot CoT (3 pts)**: Combine Approaches 2 and 3 to do 5-shot CoT prompting.

5. **Your own prompt! (5 pts)**: Try something new. Think about how you solve Math problems and implement your own prompting method.

In [10]:
def prompt_generation_zero_shot(problem: str) -> str:
    """
    Zero-shot prompt.

    Returns:
    str: The generated prompt.
    """
    # IMPLEMENT HERE
    prompt = f"Given the math problem: {problem}, solve it and conclude with Final Answer: [answer]' at the end."
    return prompt

In [11]:
def prompt_generation_zero_shot_cot(problem: str) -> str:
    """
    Zero-shot Chain of Thought (CoT) prompt.

    Returns:
    str: The generated prompt.
    """
    # IMPLEMENT HERE
    prompt = f"Let's think through this problem step by step: {problem}. Explain your reasoning step by step and conclude with 'Final Answer: [answer]' at the end."
    return prompt

In [12]:
def prompt_generation_5_shot(problem: str, training_set: Dataset) -> str:
    """
    5-shot prompt generation for GSM8K problems. Randomly selects 5 examples from the training set.

    Returns:
    str: The generated prompt with 5 in-context_examples.
    """
    examples = []

    for _ in range(5):
        random_index = random.randint(0, len(training_set['question']) - 1)
        example = {
            'question': training_set['question'][random_index],
            'processed_answer': training_set['processed_answer'][random_index]
        }
        examples.append(example)

    example_prompts = '\n'.join([f"Problem: {ex['question']}\nAnswer: {ex['processed_answer']}" for ex in examples])
    prompt = f"Solve this math problem: {problem}. Here are some examples of how to solve similar problems:{example_prompts}. Conclude with 'Final Answer: [answer]'."

    # print(prompt)
    return prompt

In [13]:
def prompt_generation_5_shot_cot(problem: str, training_set: Dataset) -> str:
    """
    5-shot Chain of Thought (CoT) prompt generation. Randomly selects 5 examples
    from the training set and includes reasoning steps.

    Returns:
    str: The generated prompt with 5 CoT in-context examples.
    """
    # IMPLEMENT HERE
    examples = []

    for _ in range(5):
        random_index = random.randint(0, len(training_set['question']) - 1)
        example = {
            'question': training_set['question'][random_index],
            'processed_answer': training_set['processed_answer'][random_index]
        }
        examples.append(example)

    example_prompts = '\n'.join([f"Problem: {ex['question']}\nReasoning: {ex['processed_answer']}" for ex in examples])

    prompt = f"Solve this math problem step by step: {problem}. Here are some examples of how to solve similar problems:{example_prompts}. Explain every step of your answer and conclude with 'Final Answer: [answer]'."

    return prompt


In [14]:
# Feel free to change the method definition

def my_prompt(problem: str) -> str:
    """
    Your own unique way of prompting an LLM for Math word problems.

    Returns:
    str: The generated prompt
    """
    # IMPLEMENT HERE
    prompt =  f"Given the problem: {problem}, solve it step by step. Start by breaking the problem into smaller parts, explain each step clearly, and conclude with the 'Final Answer: [answer]'"
    return prompt

## 3.2 Prompting Gemini and Implementing Self-Consistency (5 + 5 + 10 pts)

Here, you will help build the wrapper for prompting Gemini using the prompt methods you have designed above.

You will then also implement Self-Consistency based prompting. Refer to the [Self-Consistency Paper](https://arxiv.org/abs/2203.11171). In order to implement Self-Consistency, you generate multiple Zero-Shot CoT (Approach 2 in the prompting methods) candidates, and take a majority vote of the answers predicted by each candidate.

### First, write the function where you will process the answer generated by the model. (5 pts)

Note that answer processing changes for different prompt types, so this function also takes in the name of the method in its argument.

In [15]:
def answer_processing(prediction: str, prompt_function: Any) -> str:
    """
    Processes the model's generated output to extract the final answer.

    Returns:
    str: The processed numerical answer.
    """

    prompt_name = prompt_function.__name__
    # print("Prediction", prediction)
    # IMPLEMENT HERE
    if prompt_name == 'prompt_generation_zero_shot':
      cleaned_paragraph = re.sub(r'\.\d+', '', prediction)
      cleaned_paragraph = re.sub(r'[.,]', '', cleaned_paragraph)
      numbers = re.findall(r'\b\d+\b', cleaned_paragraph)
      answer = str(int(numbers[-1])) if numbers else None

    elif prompt_name == 'prompt_generation_zero_shot_cot':
      cleaned_paragraph = re.sub(r'\.\d+', '', prediction)
      cleaned_paragraph = re.sub(r'[.,]', '', cleaned_paragraph)
      numbers = re.findall(r'\b\d+\b', cleaned_paragraph)
      answer = str(int(numbers[-1])) if numbers else None

    elif prompt_name == 'prompt_generation_5_shot' or prompt_name == 'prompt_generation_5_shot_cot':
      cleaned_paragraph = re.sub(r'\.\d+', '', prediction)
      cleaned_paragraph = re.sub(r'[.,]', '', cleaned_paragraph)
      numbers = re.findall(r'\b\d+\b', cleaned_paragraph)
      answer = str(int(numbers[-1])) if numbers else None

    else:
      cleaned_paragraph = re.sub(r'\.\d+', '', prediction)
      cleaned_paragraph = re.sub(r'[.,]', '', cleaned_paragraph)
      numbers = re.findall(r'\b\d+\b', cleaned_paragraph)
      answer = str(int(numbers[-1])) if numbers else None

    print("Processed prediction:", answer)
    return answer

In [16]:
# Do not change, method to calculate accuracy from predictions and ground truth labels

def evaluate_accuracy(predictions: List[str], ground_truths: List[str]) -> float:
    correct = 0
    total = len(predictions)

    for pred, true in zip(predictions, ground_truths):
        if pred == true:
            correct += 1

    accuracy = correct / total
    return accuracy * 100

### Next, write the wrapper function where you use all the building blocks constructed above to prompt the Gemini model (5 + 10 pts)


On how to prompt Gemini, refer to the [Gemini Text Generation Handbook](https://ai.google.dev/gemini-api/docs/text-generation?lang=python).

Hint: Reading this will help you figure out how to generate multiple candidates to implement Self-Consistency.

In [17]:
import time
from google.api_core.exceptions import TooManyRequests
import collections
import re

def generate_with_retry(model_instance, prompt, max_retries=20, max_wait_time=120):
    attempt = 0
    while attempt < max_retries:
        try:
            # Attempt to generate the response
            response = model_instance.generate_content(prompt)
            return response
        except TooManyRequests:
            attempt += 1
            wait_time = min(2 ** attempt, max_wait_time)  # Exponential backoff with max wait time
            time.sleep(wait_time)  # Wait before retrying
    raise Exception("Max retries reached. Could not generate content.")

In [18]:
def pipeline_generate(
    model_instance: Any,
    test_set: Dataset,
    prompt_function: Callable[[str], str],
    process_answer_function: Callable[[str], str],
    evaluation_function: Callable[[List[str], List[str]], float],
    self_consistency: int,
) -> float:
    """
    Args:
    model_instance (Any): The Google Gemini model instance.
    test_set (Dataset): The GSM8K test set to evaluate on.
    prompt_function (Callable): Function to generate prompts for the test set.
    process_answer_function (Callable): Function to process the model's generated answers.
    evaluation_function (Callable): Function to evaluate model's answers against the ground truth.
    self_consistency: Number of samples to run self-consistency approach on.
    If negative, 0 or 1, this implies regular prompting

    Returns:
    float: The accuracy of the model on the test set.
    """

    predictions = []
    ground_truths = []
    train_data = process_gsm8k_answers(dataset['train'])

    for example in test_set:
        problem = example['question']
        ground_truth = example['processed_answer']
        print("ground_truth:", ground_truth)

        if prompt_function.__name__ in ['prompt_generation_5_shot','prompt_generation_5_shot_cot']:
          prompt=prompt_function(problem, train_data)
        else:
          prompt = prompt_function(problem)
        all_predictions = []

        for _ in range(self_consistency):
            try:
                response = generate_with_retry(model_instance, prompt)
                prediction = process_answer_function(response.text, prompt_function)
                all_predictions.append(prediction)
            except Exception as e:
                print(f"Error generating response: {e}")
                all_predictions.append("")


        if all_predictions:

            most_common_answer, _ = collections.Counter(all_predictions).most_common(1)[0]
            predictions.append(most_common_answer)
        else:
            predictions.append("")

        ground_truths.append(ground_truth)

    accuracy = evaluation_function(predictions, ground_truths)
    return accuracy

In [19]:
gsm8k_test_processed = process_gsm8k_answers(dataset['test'])
# The following line is just to test your systems, comment this line out to report results on the entire test set in 3.3
gsm8k_test_processed = Dataset.from_dict(gsm8k_test_processed[:50])
# print(len(gsm8k_test_processed))
# Run model generation with zero-shot prompt generation
accuracy = pipeline_generate(
    model_instance=model,
    test_set=gsm8k_test_processed,
    prompt_function=my_prompt, # Change this to test different prompt methods
    process_answer_function=answer_processing,
    evaluation_function=evaluate_accuracy,
    self_consistency=1,
)

print(f"Accuracy: {accuracy}%")

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

ground_truth: 18
Processed prediction: 18
ground_truth: 3
Processed prediction: 3
ground_truth: 70000
Processed prediction: 70000
ground_truth: 540
Processed prediction: 540
ground_truth: 20
Processed prediction: 20
ground_truth: 64
Processed prediction: 92
ground_truth: 260
Processed prediction: 260
ground_truth: 160
Processed prediction: 80
ground_truth: 45
Processed prediction: 435
ground_truth: 460
Processed prediction: 460
ground_truth: 366
Processed prediction: 366
ground_truth: 694
Processed prediction: 694
ground_truth: 13
Processed prediction: 1
ground_truth: 18
Processed prediction: 24
ground_truth: 60
Processed prediction: 60
ground_truth: 125
Processed prediction: 125
ground_truth: 230
Processed prediction: 230
ground_truth: 57500
Processed prediction: 57500
ground_truth: 7
Processed prediction: 7
ground_truth: 6
Processed prediction: 4
ground_truth: 15
Processed prediction: 14
ground_truth: 14
Processed prediction: 14
ground_truth: 7
Processed prediction: 7
ground_truth: 8

Processed prediction: 9
ground_truth: 75
Processed prediction: 75
ground_truth: 2
Processed prediction: 1
ground_truth: 10
Processed prediction: 10
ground_truth: 18
Processed prediction: 20
ground_truth: 8
Processed prediction: 8
ground_truth: 200
Processed prediction: 200
ground_truth: 26
Processed prediction: 26
ground_truth: 48
Processed prediction: 48
ground_truth: 20
Processed prediction: 20
ground_truth: 104
Processed prediction: 44
ground_truth: 163
Processed prediction: 163
ground_truth: 800
Processed prediction: 800
ground_truth: 8


Processed prediction: 8
ground_truth: 30
Processed prediction: 30
Accuracy: 78.0%


## 3.3 Complete this table based on your implementation in 3.2 and answer the following questions (5 + 5 pts)

### Round each value up to two decimal points (5 pts)

Method|Accuracy
---|---|
0-shot| 68%
0-shot CoT| 76%
5-shot| 74%
5-shot CoT| 72%
My prompt| 78%
0-shot CoT Self-Consistency| 82%

### What was the intuition behind the prompt that you designed? (2 pts)

This prompt works well because it tells the model to break the problem into smaller steps and explain each one. This helps the model think logically and avoid mistakes. It also makes the explanation clear by asking for a detailed breakdown of each step, which guides the model towards the right answer. Ending with "Final Answer: answer" ensures the model gives a clear and precise answer and it is also easier to post process the answer for evaluation. The step-by-step approach also helps the model double-check its work, improving accuracy and making the reasoning process easier to follow.

### What are the merits and demerits of using advanced prompting approaches like Chain of Thought or Self-Consistency? (3 pts)

**Merits:**

- CoT helps break down tough problems, making answers more accurate and logical. Self-consistency improves accuracy by averaging multiple answers.
- Both methods help the model deal with unclear or confusing problems by focusing on reasoning or averaging different responses.
- CoT helps the model better understand complex queries, and self-consistency helps filter out wrong answers.

**Demerits:**

- Both methods take more time and resources. CoT needs more reasoning steps, and self-consistency requires multiple answers.
- CoT might make the model focus too much on certain reasoning patterns, while self-consistency could reinforce biases. Thus there is a risk of overfitting.
- Creating good CoT prompts is hard, and it is not always clear how many responses to use for self-consistency.
